# a3 - Python

This assignment will cover topics of classification and dimensionality reduction.

Make sure that you keep this notebook named as "a3-5831.ipynb" 

Any other packages or tools, outside those listed in the assignments or Canvas, should be cleared
by Dr. Brown before use in your submission.

# Q0 - Setup

The following code looks to see whether your notebook is run on Gradescope (GS), Colab (COLAB), or the linux Python environment you were asked to setup.

In [ ]:
import re 
import os
import platform 
import sys 

# flag if notebook is running on Gradescope 
if re.search(r'amzn', platform.uname().release): 
    GS = True
else: 
    GS = False

# flag if notebook is running on Colaboratory 
try:
  import google.colab
  COLAB = True
except:
  COLAB = False

# flag if running on Linux lab machines. 
cname = platform.uname().node
if re.search(r'(guardian|colossus|c28|coc-15954-m)', cname):
    LLM = True 
else: 
    LLM = False

print("System: GS - %s, COLAB - %s, LLM - %s" % (GS, COLAB, LLM))

## Notebook Setup

It is good practice to list all imports needed at the top of the notebook. You can import modules in later cells as needed, but listing them at the top clearly shows all which are needed to be available / installed.

If you are doing development on Colab, the otter-grader package is not available, so you will need to install it with pip (uncomment the cell directly below).

In [ ]:
# Only uncomment if you developing on Colab 
# if COLAB == True: 
#     print("Installing otter:")
#     !pip install otter-grader==4.2.0 

In [ ]:
# Import standard DS packages 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy
import statistics
import textwrap
%matplotlib inline


from sklearn.model_selection import train_test_split, StratifiedKFold 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import tree        # decision tree classifier
from sklearn import neighbors   # knn classifier
from sklearn import naive_bayes # naive bayes classifier 
from sklearn import svm         # svm classifier
from sklearn import ensemble    # ensemble classifiers
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import metrics     # performance evaluation metrics
from sklearn import model_selection
from sklearn import preprocessing 
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA


# Package for Autograder 
import otter 
grader = otter.Notebook()

# Q1 - NBA Rookies

For this problem, you will go back to using the data set of rookie NBA players from 1980 - 2017 seasons.  The dataset was collected from the NBA website API - https://www.nba.com. 

You will use the data from their rookie year to predict whether a player will last at least 5 seasons in the leaque. 

The data consists of variables:

* `PlayerID`, `Player` - variables to identify individual samples (ignore for prediction)
* `Tm`, `Year` - variables describing the year the player started and for what team (ignore for prediction)  
* `TARGET` - This is the target / class feature to be predicted (whether the player was in the league for at least 5 years). 

The remaining variables are predictor variables for the models.  They come in pairs "*\_DIFF" and "*\_A" reporting the given statistic as the difference between Team A and Team B and the statistic itself for Team A. 

* `Pos` - position of the player, power forward, point guard, shooting guard, center, etc.
* `Age` - player age
* `G` - sum of number of games played
* `GS` - sum of number of games started
* `MP` - sum of number of minutes played
* `PTS` - sum of number of points scored
* `FG` - sum of number of field goals made (both 2 and 3 pointers)
* `FGA` - sum of number of field goals attempted
* `FG%` - FG / FGA, percentage of field goals made 
* `3P` - sum of the number of 3 pointers made
* `3PA` - sum of the number of 3 pointers attempted
* `3P%` - 3P / 3PA, percentage of 3 pointers made
* `2P` - sum of the number of 2 point shots made
* `2PA` - sum of the number of 2 point shots attempted
* `2P%` - 2P / 2PA, percentage of 2 point shots made
* `eFG%` - Effective Field Goal Percentage, (`FG` + 0.5 * `3P`)/`FGA`
* `FT` - sum of the number of free throws made
* `FTA` - sum of the number of free throws attempted
* `FT%` - FT / FTA, percentage of free throws made
* `ORB` - sum of the number of offensive rebounds
* `DRB` - sum of the number of defensive rebounds
* `TRB` - sum of the number of total rebounds
* `AST` - sum of the number of assists
* `STL` - sum of the number of steals
* `BLK` - sum of the number of blocks
* `TOV` - sum of the number of turnovers
* `PF` - sum of the number of personal fouls

More information on the stats used can be found: https://www.nba.com/stats/help/glossary   
*Note, some of the abbreviations used here are slightly different* 


## Q1(a) - Load Data, Handle Missing Data

**Same process as Q6(a) and Q6(c) on Assignment a2**

Load the `nba` data.

Next, you must handle the missing data. 

First, the missing values for GS, because there is not way to impute these values (and there are so few players in the 1980 and 1981 season having this information), the all samples from these two seasons should be deleted.

For the missing values in the "percentage" columns, replace those missing values with 0.

Call you new DataFrame after performing these operations nba2.


In [ ]:
nba = pd.read_csv(...) 

nba2 = ...  # nba data with missing values handles. 

nba2.head()

In [ ]:
grader.check("q1a")

## Q1(b) - Prepare the data 

**Same process as Q6(e) on Assignment a2**

At this point, we need to set up the data in order to be used in the classification models. 

We want to create a DataFrame `nbaX` for the predictor variables and `nbaY` for the target variable. 

The target variable, `nbaY` will just be the `Target` column of the `nba2` data set. 

The predictor variables have more considerations. 

We want to exclude player information such as IDs, `PlayerID` and names `Player`, that are identifying and not predictive.   

You should also exclude these other factors (note, that some of these variables may in fact be predictive but we are going to exclude at this time): 

* `POS` - player position
* `Tm` - team
* `Year` - rookie year
* `Age` - player age

Finally, several of the numeric predictive variables are not only related, but can be directly calculated from one another, e.g., `FG%` = `FG` / `FGA`.  Having variables that are closely related, or in this case redundant may actually hinder the predictive models. 

Therefore, exclude the following variables: `FG`, `3P`, `2P`, `eFG%`, `FT`, `TRB`. 

In [ ]:
nbaY = ...
nbaX = ...

print(nbaX.shape)
print(nbaY.shape)

In [ ]:
grader.check("q1b")

## Q1(c) - Model Selection and Evaluation: Three-fold Split and Scaling

**Same process as Q6(f) on Assignment a2**



Split the data into training, validation and test sets with 60, 20, and 20% of the data respectively. Make sure to split the data such that the distribution of class labels is approximately equal across splits - “stratify”.

Set the seed for the random generator in `random_state` to ”4821”


In [ ]:
# Split of the test set
X_trainval, X_test, y_trainval, y_test = ...

# Split trainval into train + val 
X_train, X_val, y_train, y_val = ...


In [ ]:
grader.check("q1c")

## Q1(d) - Support Vector Machines + GridSearch with Cross-validation (without using GridSearchCV) 

In this part, you will use the *do-it-yourself* approach using `StratifiedKFold` (rather than GridSearchCV). 

Use the same split from above with 80% train+val, 20% test data. 

With the train+val data, use 10-fold cross-validation (make sure to use Stratified approach with random_state = 4821).  Train each model on the training set and evaluate each model on the validation set.  

*Consider how to do standard scaling with this approach.  That is, think about where you fit the parameters for scaling and then transform various parts of the data set.*  

You will consider SVM models with the following hyperparameters: 

* Polynomial kernel with C = [10^-2, 10^-1, 1], degree = [1, 2, 3], with `coef0 = 1`
* RBF kernel (Gaussian kernel) with C = [10^-2, 10^-1, 1] 

Collect each model's validation performance (AUC) in `svm_poly_auc_val` and `svm_rbf_auc_val` variables.  

Report the mean validation performance (AUC) as DataFrame with: 

- rows, Linear kernel, poly kernel d=2, poly kernel d=3, rbf kernel 
- columns, C = [10^-2, 10^-1, 1]

Report the best parameter combination (cost + kernel). 

Retrain the best model on train+val and report the test performance.  *Make sure to scale the train+val and test data, like it was done in Q6(g) in Assignment a2.*

In [ ]:

svm_poly_auc_val = np.zeros((10,3,3))
svm_rbf_auc_val = np.zeros((10,3))


# With the train+val data, use 10-fold cross-validation (with StratifiedKFold )  

# Train each model on the train set, evaluate each model on the validation set
#  set up scaling 

# You will consider SVM models with the following hyperparameters: 
#   - Polynomial kernel with C = [10^-2, 10^-1, 1], degree = [1, 2, 3]
#   - RBF kernel (Gaussian kernel) with C = [10^-2, 10^-1, 1] 

# Collect each model's validation performance. 


# Report the mean validation performance (AUC) as DataFrame with: 
#   - rows, Linear kernel, poly kernel d=2, poly kernel d=3, rbf kernel 
#   - columns, C = [10^-2, 10^-1, 1]

svm_results = ...

# Report the best hyperparameter combination (cost + kernel). 
svm_bestC = ...  # numeric value
svm_bestKernel = ...  # string `poly` or `rbf` 
svm_bestD = ... # integer or np.nan

# Retrain the best model on train+val and report the test performance .
#  Scale train+val and test data like Q6(g) in Assignment a2 
scaler_final = ...
X_test_sc = ...
X_trainval_sc = ...


svm_auc_test = ...

print('best hyperparameters: %d, %s, %d' % (svm_bestC, 
                                            svm_bestKernel, svm_bestD))
svm_results

In [ ]:
grader.check("q1d")

## Q1(e) -  Ensemble Methods + GridSearchCV with Pipelines

Let’s examine bagging & boosting ensemble approaches for prediction.

For this part, we will use the **preferred method for building predictor models by using pipelines**. 

You will create a pipeline for the Random Forest models and a pipeline for the AdaBoost models. Both pipelines will use standard scaling preprocessing.  Label the steps of the pipeline with "sc" for the scaling step and "rf" and "ab" for the modeling step in the two different pipelines.

For the random forest, consider hyper-parameters for the maximum number of features: [2, 4, 8, 16] and number of estimators of [25, 50, 100].  For AdaBoost, consider the hyper-parameter of the number of estimators as [10, 25, 50, 100]. 

To ensure repeatability or your code (and to compare to the autograder) make sure to set the random state in both classifiers and the stratified 10-fold cross-validation to "4821".  

Use AUC as the scoring metric for the GridSearch criteria.  

You will need to report the best hyper-parameters for both models as well as the final test set performance. 


In [ ]:

# You will create a pipeline for both the Random Forest and AdaBoost models. 
#   Both pipelines will use standard scaling preprocessing. 
#   Call the scaling step "sc" and model step for each pipeline, "rf" / "ab" 
#   Set the random state in both classifiers to 4821
ab_pipe = ...

rf_pipe = ... 

# RF: hyper-parameters for the maximum number of features: [2, 4, 8, 16] and 
#   number of estimators of [25, 50, 100].  
rf_params = ...

# AdaBoost: hyper-parameter of the number of estimators as [10, 25, 50, 100].
ab_params = ...

# Set the random state in the stratified k-fold cv to 4821  
cvStrat = ... 

# Use AUC as the scoring metric for the GridSearch criteria. 
rf_grid = ... 

ab_grid = ...

# Report the best hyper-parameters and final test set performance 
rf_best_params = ...
ab_best_params = ...


rf_auc_test = ...
ab_auc_test = ...


In [ ]:
grader.check("q1e")

## Q1(f) - Feature Selection 

Now that you have explored several different classification models and how to create pipelines, let's explore create a pipeline that considers different scaling methods, feature selection options, and models in a pipeline. 

Specifically, you will create a pipeline that considers the following steps using the labels below. 

* Scaling - `scaler`
    * MinMaxScaling with default values
    * StandardScaling with the default values
* Feature Selection - `feat_sel`
    * Use `SelectPercentile` as a way to select features
    * For the `SelectPercentile` consider 10, 25, 50, 75, and 100% of your data, use the `f_classif` option to select the features
* Classification Models - `classify`
    * Consider a KNN model with k = [11, 15, 19]
    * Consider a Random Forest model with number of trees [10, 25, 50] and max_features in [2, 8, 16]
    * Consider an AdaBoost model with number of estimators of [10, 25, 50]
 
To ensure repeatability or your code (and to compare to the autograder) make sure to set the random state in the Random Forest and AdaBoost classifiers and the stratified 10-fold cross-validation to "4821".  

Use AUC as the scoring metric for the GridSearch criteria.  

You will need to report the best hyper-parameters as well as the final test set performance. 


In [ ]:

#  you will create a pipeline that considers the following steps
#    - Scaling - labeled `scaler`
#    - Feature Selection - labeled `feat_sel`
#    - Classification model - labeled `classify`

final_pipe = ... 

# Consider the following hyperparameters for each step:
#  (you may need to create a list of dictionaries to store the options) 
# * Scaling - `scaler`
#     * MinMaxScaling with default values
#     * StandardScaling with the default values
# * Feature Selection - `feat_sel`
#     * Use `SelectPercentile` as a way to select features
#     * For the `SelectPercentile` consider 10, 25, 50, 75, and 100% of the data
#         use the `f_classif` option to select the features
# * Classification Models - `classify`
#     * Consider a KNN model with k = [11, 15, 19]
#     * Consider a Random Forest model with number of trees [10, 25, 50] and 
#          max_features in [2, 5, 10, 15]
#     * Consider an AdaBoost model with number of estimators of [10, 25, 50]

final_params = ...


# Set the random state in the stratified k-fold cv to 4821  
cvStrat = ... 

# Use AUC as the scoring metric for the GridSearch criteria. 
final_grid = ... 


# Report the best hyper-parameters and final test set performance 
final_best_params = ...

final_auc_test = ...

print(final_best_params)
print(final_auc_test)

In [ ]:
grader.check("q1f")

## Q1(g) - Feature Transformation with PCA 

For this section, you will create a new pipeline that considers using PCA as a method for feature transformation and selection. 

In the pipeline, you will have three steps for scaling the data using `StandardScaler` labeled `scaler`, a step to run PCA and select the number of components to use labeled `feat_trans`, and the classification step using the `RandomForestClassifier` labeled `classify`. 

The hyperparameters to tune in the pipeline are: 

* PCA step 
    * number of components to keep: [2, 4, 8, 16]
* Random Forest 
    * number of estimators: [10, 25, 50] 
    * maximum number of features: [2, 8, 16]

To ensure repeatability or your code (and to compare to the autograder) make sure to set the random state in the Random Forest classifier and the stratified 10-fold cross-validation to "4821".

Use AUC as the scoring metric for the GridSearch criteria.

You will need to report the best hyperparameters as well as the final test set performance.

In [ ]:

#  you will create a pipeline that uses PCA as a feature transformation step
#    - Scaling - labeled `scaler` (with standard parameters)
#    - Feature Transformation - labeled `feat_trans`
#    - Classification model - labeled `classify`

pca_pipe = ... 

# Consider the following hyperparameters for each step:
# * Feature Transformation - `feat_trans`
#     * number of components: [2, 4, 8, 16]
# * Classification Models - `classify`
#     * Consider a Random Forest model with number of trees [10, 25, 50] and 
#          max_features in [2, 5, 10, 15]

pca_params = ...


# Set the random state in the stratified k-fold cv to 4821  
cvStrat = ... 

# Use AUC as the scoring metric for the GridSearch criteria. 
pca_grid = ... 


# Report the best hyper-parameters and final test set performance 
pca_best_params = ...

pca_auc_test = ...

print(pca_best_params)
print(pca_auc_test)

In [ ]:
grader.check("q1g")

# Q2 - Pokemon Data 

Consider the following data set of Pokemon (collected from several sources, pokemon.com, pokemondb, bulbapedia). 



## Q2(a) - Load the data 

Load the Pokemon data into a Data Frame, `pok`, consider how to read in missing data.

In [ ]:
# Read in Pokemon data

pok = pd.read_csv(.....)

pok.head()

In [ ]:
grader.check("q2a")

<!-- BEGIN QUESTION -->

## Q2(b) - Explore the Data 

Look at the distribution of Pokemon skills we will use in this analysis: `hp`, `attack`, `defense`, `sp_attack` - Special Attack, `sp_defense` - Special Defense, and `speed`. 

Create a violinplot showing the distribution of these variables. 


In [ ]:
# Create violinplot




<!-- END QUESTION -->

## Q2(c) - Prepare the data 

Let's use the 6 pokemon skills viewed above.  

The six features (Pokemon Skills) have different ranges, therefore we should scale the data before considering PCA. Standardize the data over the six features of interest, store only these variables in `pokScaled` DataFrame.

In [ ]:
# Scale the data 

pokScaled = ...


In [ ]:
grader.check("q2c")

## Q2(d) - Principal Components Analysis 

Perform principal components analysis (PCA) on the scaled Pokemon skills data. 
Store the results of PCA in `pokPCA` and the transformed data in `pokTr`.

In [ ]:
# Run PCA on the scaled data 

pokPCA = ...
pokTr = ...


In [ ]:
grader.check("q2d")

<!-- BEGIN QUESTION -->

## Q2(e) - Explore the Results 

Plot the transformed data in the space defined by the first two principal components. This should be a scatterplot.

In [ ]:
# Plot the transformed data  




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q2(f) - Explore the Results, part 2 

Explore the amount of variance explained by each principal component direction. Plot the proportion of variance explained (y-axis) vs. the different principal components.

In [ ]:
# Plot variance explained by each component   




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q2(g) - Explore the Results, part 3

Plot the cumulative proportion of variance explained by the principal components.


In [ ]:
# Plot the cumulative variance explained.   




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q2 bonus 

Display the transformed data in the space defined by the first two principal components.  Let's also color the data points by `type1`.  Place the legend outside the plot to the right. 



In [ ]:
# Create plot   




<!-- END QUESTION -->

# Q3 - Polarization of Congress 

For this part of the project you will consider methods to group members of U.S. House of Representatives based on their voting records. The voting records from congress are available at Office of the Clerk, US House of Representatives, but not in a form that is easily digestible for analysis.

In fact it was only in 2016, that Congress agreed to make legislative data available themselves. Govtrack.us has links to primary data sources and api’s projects that collect and release the data in easier digestible forms.

A long-standing project to document congressional roll call votes at the Interuniversity Consortium for Political and Social Research (ICPSR). This data includes roll call votes from 1789 - 1990. The ICPSR formatting for storing this data has been used on other sites which are keeping up with the creating a record.

The data you will use was downloaded in its raw state from https://voteview.com/data. Then, for each of the last 20 congresses (up until 2021), 50 random votes were selected for each member (ignoring the first 10 votes of each session - these votes usually have to do with electing a Speaker of the House and rules votes). The results are stored in the the files `H97_votes.csv`, `H98_votes.csv`, ..., `H116_votes.csv`.


References: 
- http://clerk.house.gov/legislative/legvotes.aspx
- https://www.icpsr.umich.edu/icpsrweb/ICPSR/series/159

## Q3(a) - Load and prepare the data 

Working with data for the 116th Congress, H116 votes.csv, read in the vote data. 

Preprocess the data appropriately. 



In [ ]:
# Read in votes data into DataFrame 

votes = ...

# prepare the data for PCA 
# Scaled votes data only
votes_sc = ...



In [ ]:
grader.check("q3a")

## Q3(b) - PCA 

Perform principal component analysis on the voting records.  

Plot the first two principal components. Because we know the party affiliation of each member of congress, color the plot based on party (red = Republican, PartyCode=200; blue = Democrat, PartyCode = 100; green = Independent, PartyCode in 300s).

In [ ]:

# Run PCA and transform the data 

votes_pca = ...
votes_tr = ...

# Create plot


In [ ]:
grader.check("q3b")

<!-- BEGIN QUESTION -->

## Q3(c) - Historical Polarization (bonus)

Create a small multiples plot (6 x 3) showing the results of PCA (colored by party) for the 18 Congresses starting with the 97th.

In [ ]:
# Create small multiples plot



<!-- END QUESTION -->



## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

**NOTE** the submission must be run on the campus linux machines.  See the instruction in the Canvas assignment.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)